In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Installing FastBook**

In [ ]:
pip install fastbook

In [2]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
#hide
from fastbook import *
from fastai.vision.widgets import *

# **Using Bing Search API**

In [4]:
key = os.environ.get('AZURE_SEARCH_KEY', 'd7e104dd949f4e199834cc3c4ac1b94f')

In [5]:
search_images_bing

# **Gathering & Cleaning Data**

In [6]:
results=search_images_bing(key,"ambulance")
ims=results.attrgot('contentUrl')
len(ims)

In [7]:
ims=['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRoLw47urRRDmFk0v7yZWPfFRVZcSnTC8ouZo-gvQJcP5l74FYmmnlbifc8cZ47VRcZx7k&usqp=CAU']

In [8]:
dest='images/ambulance.jpg'
download_url(ims[0],dest)

In [9]:
im=Image.open(dest)
im.to_thumb(128,128)

In [10]:
emergency_vehicle_types='ambulance','police_mobile','fire_truck'
path=Path('Emergency')

In [11]:
if not path.exists():
    path.mkdir()
    for o in emergency_vehicle_types:
        dest =(path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key,f'{o} emergency')
        download_images(dest,urls=results.attrgot('contentUrl'))

In [13]:
fns= get_image_files(path)
fns

In [14]:
failed=verify_images(fns)
failed

In [15]:
failed.map(Path.unlink);

In [ ]:
??verify_images

# **DataBlock and Image Resizing**

In [16]:
emergency=  DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [17]:
dls = emergency.dataloaders(path)

In [18]:
dls.valid.show_batch(max_n=8,nrows=1)

# **Squish**

In [19]:
emergency= emergency.new(item_tfms=Resize(128,ResizeMethod.Squish))
dls=emergency.dataloaders(path)
dls.valid.show_batch(max_n=8,nrows=1)

In [20]:
emergency= emergency.new(item_tfms=Resize(128,ResizeMethod.Pad,pad_mode='zeros'))
dls=emergency.dataloaders(path)
dls.valid.show_batch(max_n=8,nrows=1)

# **Resizing Randomly**

In [21]:
emergency= emergency.new(item_tfms=RandomResizedCrop(128,min_scale=0.3))
dls=emergency.dataloaders(path)
dls.train.show_batch(max_n=8,nrows=1,unique=True)

# **Data Augmentation**

In [22]:
emergency = emergency.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = emergency.dataloaders(path)
dls.train.show_batch(max_n=16, nrows=2, unique=True)

# **Training Your Model, and Using It to Clean Your Data**

In [23]:
emergency=emergency.new(
item_tfms=RandomResizedCrop(224,min_scale=0.5),
batch_tfms=aug_transforms())
dls=emergency.dataloaders(path)

In [24]:
learn=vision_learner(dls,resnet18,metrics=error_rate)
learn.fine_tune(4)

In [25]:
inter=ClassificationInterpretation.from_learner(learn)
inter.plot_confusion_matrix()

In [26]:
inter.plot_top_losses(3,nrows=1)

**Deleting & Re-labelling**

In [27]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [28]:
for idx in cleaner.delete():
    cleaner.fns[idx].unlink()

In [29]:
for idx.cat in cleaner.change():
    shutil.move(str(cleaner.fns[idx]), path/cat)

# **Turning Your Model into an Online Application**

**Using the Model for Inference**

In [30]:
learn.export()

In [31]:
path=Path()
path.ls(file_exts='.pkl')

In [32]:
learn_inf=load_learner(path/'export.pkl')

In [33]:
learn_inf.predict('images/ambulance.jpg')

In [34]:
learn_inf.dls.vocab

In [35]:
btn_upload = widgets.FileUpload()
btn_upload

In [37]:
img=PILImage.create(btn_upload.data[-1])

In [38]:
out_pl=widgets.Output()
out_pl.clear_output()
with out_pl:
    display(img.to_thumb(128,128))
    out_pl

In [39]:
pred,pred_idx,probs = learn_inf.predict(img)

In [40]:
lbl_pred=widgets.Label()
lbl_pred.value=f'Prediction:{pred}; Probablity: {probs[pred_idx]:.04f}'
lbl_pred

In [41]:
btn_run = widgets.Button(description='Classify')
btn_run

In [42]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value=f'Prediction:{pred}; Probablity: {probs[pred_idx]:.04f}'
btn_run.on_click(on_click_classify)

In [43]:
btn_upload=widgets.FileUpload()

In [44]:
VBox([widgets.Label('Select Your Vehicle'),
     btn_upload,btn_run,out_pl,lbl_pred])

# **Turning Your Notebook into a Real App**

In [47]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila